## 과제 목표
1. 코사인 유사도 적용: 유클리드 거리 대신 코사인 유사도를 사용하여 사용자 간의 유사도
를 계산하고, 이를 바탕으로 사용자 0에게 영화 추천을 수행합니다.
2. 추천 결과 비교: 2가지 방법(유클리드 거리, 코사인 유사도)의 추천 결과를 비교하고 차이
를 분석합니다.
3. (선택 사항): 수업에서 만들었던 협업 필터링 알고리즘을 더 보완할 방법을 생각해보고, 
적용해서 결과를 비교해보세요. 개선한 알고리즘은 어떻게 개선했는지 주석으로 남겨주세
요

### 세부 지침
1. 코사인 유사도 적용
    - sklearn.metrics.pairwise 모듈의 cosine_similarity 함수를 사용하여 사용자 간의 코사인 유사도를 계산합니다.
    - 코사인 유사도를 사용하여 각 사용자의 k-최근접 이웃을 찾습니다 (k=5 유지).
    - 사용자 0에 대해 예측 평점을 계산하고, 상위 5개의 영화를 추천합니다.
3. 추천 결과 비교 및 분석
    - 각 방법에 따른 추천 목록의 차이를 분석하고, 어떤 방법이 더 나은 결과를 제공하는지에 대해 자신의 의견을 서술합니다.
    - 추천된 영화의 다양성, 예상 평점 등을 고려하여 비교합니다

In [3]:
import pandas as pd
import numpy as np

ratings = pd.read_csv('data/ratings.csv')
df = ratings.pivot_table(index = 'userId', columns = 'movieId', values = 'rating', aggfunc = 'sum').values
df

array([[4. , nan, 4. , ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [2.5, 2. , 2. , ..., nan, nan, nan],
       [3. , nan, nan, ..., nan, nan, nan],
       [5. , nan, nan, ..., nan, nan, nan]])

In [8]:
movies = pd.read_csv('data/movies.csv')
movie_names = {}

for i in range(len(movies)):
    a = movies.iloc[i]
    movie_names[a['movieId']] = a['title']

In [4]:
nan_mask = np.isnan(df) #누락값 있는 곳 미리 저장

df_filled = df.copy()

for i in range(len(df_filled)):
    mean = np.nanmean(df_filled[i]) #각각의 유저들의 평균값
    df_filled[i] = np.nan_to_num(df_filled[i], nan = mean)
    df_filled[i] -= mean
df_filled

array([[-0.36637931,  0.        , -0.36637931, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.63417569, -1.13417569, -1.13417569, ...,  0.        ,
         0.        ,  0.        ],
       [-0.27027027,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.31144393,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

## 유클리드 거리

In [5]:
from sklearn.metrics.pairwise import euclidean_distances

#유클리드 거리 계산
distance_matrix = euclidean_distances(df_filled)
distance_matrix

array([[ 0.        , 12.88018162, 17.71367592, ..., 32.52330461,
        12.52374725, 33.10732342],
       [12.88018162,  0.        , 13.57431978, ..., 31.40943515,
         5.18205377, 31.11334478],
       [17.71367592, 13.57431978,  0.        , ..., 33.81269431,
        13.1676513 , 33.27108605],
       ...,
       [32.52330461, 31.40943515, 33.81269431, ...,  0.        ,
        31.07357893, 42.64413263],
       [12.52374725,  5.18205377, 13.1676513 , ..., 31.07357893,
         0.        , 31.07797619],
       [33.10732342, 31.11334478, 33.27108605, ..., 42.64413263,
        31.07797619,  0.        ]])

In [6]:
# 고려할 이웃의 수
k = 5 #다섯명정도만 골라봄.

#각 유저별로 가장 거리가 짧았던 5명이 있는 위치
nearest_neighbors = np.argsort(distance_matrix, axis = 1)[:, 1:k+1] #자기 자신을 빼고 5명 고려
nearest_neighbors

array([[ 52,  48, 188, 213, 514],
       [ 52, 188,  48, 514,  24],
       [ 52,  48, 514, 188,  24],
       ...,
       [315, 430, 539,  71,  53],
       [ 52,  48,  53, 514, 188],
       [462, 361, 347, 292, 121]], dtype=int64)

In [16]:
def movie_reco(user):
    user_ratings = df[user] #해당 유저의 데이터
    user_nan_mask = nan_mask[user] #해당 유저의 누락값이 어디 있는지
    neighbor_indices = nearest_neighbors[user] #유저들과 가장 가까운 데이터
    unrated_indices = np.where(user_nan_mask)[0] #평가하지 않았던 영화의 인덱스

    predictions = {} # 영화 별예측된 평점들을 저장해놓을 장소

    for movie_idx in unrated_indices:#평가하지 않은 영화의 인덱스를 하나씩 가져옴
        neighbor_ratings = [] #이웃들이 이 영화를 몇점을 평가했는가를 저장
        for neighbor_idx in neighbor_indices:
            if not (nan_mask[neighbor_idx][movie_idx]):
                neighbor_ratings.append(df[neighbor_idx][movie_idx])
        if neighbor_ratings != []: #비어있지 않다면
            predictions[movie_idx] = np.mean(neighbor_ratings)

    #키와 벨류값 묶어서 정렬
    sorted_predictions = sorted(predictions.items(), key = lambda i : i[1], reverse =True) 
    for i in sorted_predictions[:5]:
        print(f'추천영화 : {movie_names[i[0]]} (예측평점 : {i[1]}점)')
movie_reco(0)

추천영화 : Jeffrey (1995) (예측평점 : 5.0점)
추천영화 : Burnt by the Sun (Utomlyonnye solntsem) (1994) (예측평점 : 5.0점)
추천영화 : Virtuosity (1995) (예측평점 : 5.0점)
추천영화 : Four Weddings and a Funeral (1994) (예측평점 : 5.0점)
추천영화 : Beverly Hillbillies, The (1993) (예측평점 : 5.0점)


## 코사인 유사도

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

#코사인 유사도 계산
distance_matrix2 = cosine_similarity(df_filled)
distance_matrix2

array([[ 1.00000000e+00,  1.26451574e-03,  5.52577176e-04, ...,
         7.52238457e-02, -2.57125541e-02,  1.09323166e-02],
       [ 1.26451574e-03,  1.00000000e+00,  0.00000000e+00, ...,
        -6.00082818e-03, -6.00909967e-02,  2.49992083e-02],
       [ 5.52577176e-04,  0.00000000e+00,  1.00000000e+00, ...,
        -1.30006374e-02,  0.00000000e+00,  1.95499646e-02],
       ...,
       [ 7.52238457e-02, -6.00082818e-03, -1.30006374e-02, ...,
         1.00000000e+00,  5.07144903e-02,  5.44538770e-02],
       [-2.57125541e-02, -6.00909967e-02,  0.00000000e+00, ...,
         5.07144903e-02,  1.00000000e+00, -1.24714266e-02],
       [ 1.09323166e-02,  2.49992083e-02,  1.95499646e-02, ...,
         5.44538770e-02, -1.24714266e-02,  1.00000000e+00]])

In [18]:
# 고려할 이웃의 수
k = 5 #다섯명정도만 골라봄.

#각 유저별로 가장 거리가 짧았던 5명이 있는 위치
nearest_neighbors2 = np.argsort(distance_matrix2, axis = 1)[:, 1:k+1] #자기 자신을 빼고 5명 고려
nearest_neighbors2

array([[500, 369,  70, 394, 556],
       [510,  53, 549, 211, 169],
       [313, 182,  59,  83, 172],
       ...,
       [425, 516, 415, 597,   9],
       [207, 172,  48, 209, 394],
       [147, 550, 172,  46,  42]], dtype=int64)

In [51]:
def movie_reco2(user):
    user_ratings = df[user] #해당 유저의 데이터
    user_nan_mask = nan_mask[user] #해당 유저의 누락값이 어디 있는지
    neighbor_indices = nearest_neighbors2[user] #유저들과 가장 가까운 데이터
    unrated_indices = np.where(user_nan_mask)[0] #평가하지 않았던 영화의 인덱스

    predictions = {} # 영화 별예측된 평점들을 저장해놓을 장소

    for movie_idx in unrated_indices:#평가하지 않은 영화의 인덱스를 하나씩 가져옴
        neighbor_ratings = [] #이웃들이 이 영화를 몇점을 평가했는가를 저장
        for neighbor_idx in neighbor_indices:
            if not (nan_mask[neighbor_idx][movie_idx]):
                neighbor_ratings.append(df[neighbor_idx][movie_idx])
        if neighbor_ratings != []: #비어있지 않다면
            predictions[movie_idx] = np.mean(neighbor_ratings)

    #키와 벨류값 묶어서 정렬
    sorted_predictions = sorted(predictions.items(), key = lambda i : i[1], reverse =True) 
    for i in sorted_predictions[:5]:
        if i[0] in movie_names.keys():
            print(f'추천영화 : {movie_names[i[0]]} (예측평점 : {i[1]}점)')
        else:
            print(f"추천영화: {i[0]} 이름 미상 (예측평점 : {i[1]}점)")
    
movie_reco2(0)

추천영화 : Sense and Sensibility (1995) (예측평점 : 5.0점)
추천영화 : Anne Frank Remembered (1995) (예측평점 : 5.0점)
추천영화: 630 이름 미상 (예측평점 : 5.0점)
추천영화 : Emma (1996) (예측평점 : 5.0점)
추천영화 : To Gillian on Her 37th Birthday (1996) (예측평점 : 5.0점)


#### 유클리드는 상대적 차이에 민감, 코사인 유사도의 경우에는 평점의 크기보다는 평점의 패턴에 집중 
#### 즉, 서로다른 것을 추천하는 것으로 보임.

# 둘을 섞는다면?

In [56]:
enu = 1 / (1 + distance_matrix) #유클리드 역수

hybrid = 0.5 * enu + 0.5 * distance_matrix2

# 고려할 이웃의 수
k = 5 #다섯명정도만 골라봄.

#각 유저별로 가장 거리가 짧았던 5명이 있는 위치
nearest_neighbors3 = np.argsort(hybrid, axis = 1)[:, 1:k+1] #자기 자신을 빼고 5명 고려
nearest_neighbors3

array([[500, 369,  70,  38, 410],
       [413, 304,  62, 211, 351],
       [435, 367, 313,  83, 605],
       ...,
       [425, 516,   9, 415, 597],
       [272, 209, 186,  88, 159],
       [147, 550, 172,  46, 364]], dtype=int64)

In [58]:
def movie_reco_hybrid(user):
    user_ratings = df[user] #해당 유저의 데이터
    user_nan_mask = nan_mask[user] #해당 유저의 누락값이 어디 있는지
    neighbor_indices = nearest_neighbors3[user] #유저들과 가장 가까운 데이터
    unrated_indices = np.where(user_nan_mask)[0] #평가하지 않았던 영화의 인덱스

    predictions = {} # 영화 별예측된 평점들을 저장해놓을 장소

    for movie_idx in unrated_indices:#평가하지 않은 영화의 인덱스를 하나씩 가져옴
        neighbor_ratings = [] #이웃들이 이 영화를 몇점을 평가했는가를 저장
        for neighbor_idx in neighbor_indices:
            if not (nan_mask[neighbor_idx][movie_idx]):
                neighbor_ratings.append(df[neighbor_idx][movie_idx])
        if neighbor_ratings != []: #비어있지 않다면
            predictions[movie_idx] = np.mean(neighbor_ratings)

    #키와 벨류값 묶어서 정렬
    sorted_predictions = sorted(predictions.items(), key = lambda i : i[1], reverse =True) 
    for i in sorted_predictions[:5]:
        if i[0] in movie_names.keys():
            print(f'추천영화 : {movie_names[i[0]]} (예측평점 : {i[1]}점)')
        else:
            print(f"추천영화: {i[0]} 이름 미상 (예측평점 : {i[1]}점)")
    
movie_reco_hybrid(0)

추천영화 : Sense and Sensibility (1995) (예측평점 : 5.0점)
추천영화 : Anne Frank Remembered (1995) (예측평점 : 5.0점)
추천영화 : Chungking Express (Chung Hing sam lam) (1994) (예측평점 : 5.0점)
추천영화 : Wild Bill (1995) (예측평점 : 5.0점)
추천영화: 443 이름 미상 (예측평점 : 5.0점)


# 이웃이 3명이상인 것만 추출

In [66]:
def movie_reco_hybrid_3(user):
    user_ratings = df[user] #해당 유저의 데이터
    user_nan_mask = nan_mask[user] #해당 유저의 누락값이 어디 있는지
    neighbor_indices = nearest_neighbors3[user] #유저들과 가장 가까운 데이터
    unrated_indices = np.where(user_nan_mask)[0] #평가하지 않았던 영화의 인덱스

    predictions = {} # 영화 별예측된 평점들을 저장해놓을 장소

    for movie_idx in unrated_indices:#평가하지 않은 영화의 인덱스를 하나씩 가져옴
        neighbor_ratings = [] #이웃들이 이 영화를 몇점을 평가했는가를 저장
        for neighbor_idx in neighbor_indices:
            if not (nan_mask[neighbor_idx][movie_idx]):
                neighbor_ratings.append(df[neighbor_idx][movie_idx])
        if len(neighbor_ratings) >=3 : #비어있지 않다면
            print(neighbor_ratings)
            predictions[movie_idx] = np.mean(neighbor_ratings)

    #키와 벨류값 묶어서 정렬
    sorted_predictions = sorted(predictions.items(), key = lambda i : i[1], reverse =True) 
    for i in sorted_predictions[:5]:
        if i[0] in movie_names.keys():
            print(f'추천영화 : {movie_names[i[0]]} (예측평점 : {i[1]}점)')
        else:
            print(f"추천영화: {i[0]} 이름 미상 (예측평점 : {i[1]}점)")
    
movie_reco_hybrid_3(0)

[4.0, 3.0, 3.0]
[3.0, 4.0, 5.0]
[4.0, 3.0, 4.0, 3.0]
추천영화 : Georgia (1995) (예측평점 : 4.0점)
추천영화 : Perfect World, A (1993) (예측평점 : 3.5점)
추천영화 : Heat (1995) (예측평점 : 3.3333333333333335점)


## 둘의 교집합을 구한다면?

In [77]:
def reco_enu(user):
    user_ratings = df[user] #해당 유저의 데이터
    user_nan_mask = nan_mask[user] #해당 유저의 누락값이 어디 있는지
    neighbor_indices = nearest_neighbors[user] #유저들과 가장 가까운 데이터
    unrated_indices = np.where(user_nan_mask)[0] #평가하지 않았던 영화의 인덱스

    predictions = {} # 영화 별예측된 평점들을 저장해놓을 장소

    for movie_idx in unrated_indices:#평가하지 않은 영화의 인덱스를 하나씩 가져옴
        neighbor_ratings = [] #이웃들이 이 영화를 몇점을 평가했는가를 저장
        for neighbor_idx in neighbor_indices:
            if not (nan_mask[neighbor_idx][movie_idx]):
                neighbor_ratings.append(df[neighbor_idx][movie_idx])
        if len(neighbor_ratings) >=2: #비어있지 않다면
            predictions[movie_idx] = np.mean(neighbor_ratings)

    #키와 벨류값 묶어서 정렬
    sorted_predictions = sorted(predictions.items(), key = lambda i : i[1], reverse =True) 
    for i in sorted_predictions:
        if i[0] in movie_names.keys():
            print(f'추천영화 : {movie_names[i[0]]} (예측평점 : {i[1]}점)')
        else:
            print(f"추천영화: {i[0]} 이름 미상 (예측평점 : {i[1]}점)")
            
    return sorted_predictions

In [78]:
def reco_cos(user):
    user_ratings = df[user] #해당 유저의 데이터
    user_nan_mask = nan_mask[user] #해당 유저의 누락값이 어디 있는지
    neighbor_indices = nearest_neighbors2[user] #유저들과 가장 가까운 데이터
    unrated_indices = np.where(user_nan_mask)[0] #평가하지 않았던 영화의 인덱스

    predictions = {} # 영화 별예측된 평점들을 저장해놓을 장소

    for movie_idx in unrated_indices:#평가하지 않은 영화의 인덱스를 하나씩 가져옴
        neighbor_ratings = [] #이웃들이 이 영화를 몇점을 평가했는가를 저장
        for neighbor_idx in neighbor_indices:
            if not (nan_mask[neighbor_idx][movie_idx]):
                neighbor_ratings.append(df[neighbor_idx][movie_idx])
        if neighbor_ratings != []: #비어있지 않다면
            predictions[movie_idx] = np.mean(neighbor_ratings)

    #키와 벨류값 묶어서 정렬
    sorted_predictions = sorted(predictions.items(), key = lambda i : i[1], reverse =True) 
    for i in sorted_predictions:
        if i[0] in movie_names.keys():
            print(f'추천영화 : {movie_names[i[0]]} (예측평점 : {i[1]}점)')
        else:
            print(f"추천영화: {i[0]} 이름 미상 (예측평점 : {i[1]}점)")
    
    return sorted_predictions

In [85]:
enu_re = reco_enu(0)
cos_re = reco_cos(0)

추천영화: 6693 이름 미상 (예측평점 : 5.0점)
추천영화: 9445 이름 미상 (예측평점 : 4.75점)
추천영화: 7355 이름 미상 (예측평점 : 4.666666666666667점)
추천영화 : Malibu's Most Wanted (2003) (예측평점 : 4.5점)
추천영화 : Miracle on 34th Street (1994) (예측평점 : 4.333333333333333점)
추천영화 : Empire (2002) (예측평점 : 4.25점)
추천영화: 7750 이름 미상 (예측평점 : 4.25점)
추천영화 : Sense and Sensibility (1995) (예측평점 : 5.0점)
추천영화 : Anne Frank Remembered (1995) (예측평점 : 5.0점)
추천영화: 630 이름 미상 (예측평점 : 5.0점)
추천영화 : Emma (1996) (예측평점 : 5.0점)
추천영화 : To Gillian on Her 37th Birthday (1996) (예측평점 : 5.0점)
추천영화 : Chungking Express (Chung Hing sam lam) (1994) (예측평점 : 4.5점)
추천영화 : Jane Eyre (1996) (예측평점 : 4.5점)
추천영화: 793 이름 미상 (예측평점 : 4.5점)
추천영화: 1134 이름 미상 (예측평점 : 4.5점)
추천영화 : Annie Hall (1977) (예측평점 : 4.5점)
추천영화 : Journey of Natty Gann, The (1985) (예측평점 : 4.5점)
추천영화 : Idiots, The (Idioterne) (1998) (예측평점 : 4.5점)
추천영화 : Road Trip (2000) (예측평점 : 4.5점)
추천영화 : On Her Majesty's Secret Service (1969) (예측평점 : 4.5점)
추천영화 : Little Nicky (2000) (예측평점 : 4.5점)
추천영화 : Making Mr. Right (1987) (예측평점

In [83]:
enu_re

[(6693, 5.0),
 (9445, 4.75),
 (7355, 4.666666666666667),
 (6298, 4.5),
 (277, 4.333333333333333),
 (5901, 4.25),
 (7750, 4.25)]

In [86]:
cos_re

[(17, 5.0),
 (116, 5.0),
 (630, 5.0),
 (838, 5.0),
 (1043, 5.0),
 (123, 4.5),
 (613, 4.5),
 (793, 4.5),
 (1134, 4.5),
 (1230, 4.5),
 (2077, 4.5),
 (3569, 4.5),
 (3617, 4.5),
 (3633, 4.5),
 (3979, 4.5),
 (4131, 4.5),
 (4755, 4.5),
 (4791, 4.5),
 (138, 4.333333333333333),
 (3189, 4.25),
 (77, 4.0),
 (144, 4.0),
 (145, 4.0),
 (277, 4.0),
 (297, 4.0),
 (302, 4.0),
 (326, 4.0),
 (454, 4.0),
 (455, 4.0),
 (514, 4.0),
 (522, 4.0),
 (545, 4.0),
 (607, 4.0),
 (622, 4.0),
 (629, 4.0),
 (633, 4.0),
 (649, 4.0),
 (658, 4.0),
 (983, 4.0),
 (986, 4.0),
 (1052, 4.0),
 (1444, 4.0),
 (1972, 4.0),
 (2601, 4.0),
 (2992, 4.0),
 (3557, 4.0),
 (4421, 4.0),
 (4607, 4.0),
 (4900, 4.0),
 (5250, 4.0),
 (5363, 4.0),
 (5379, 4.0),
 (5834, 4.0),
 (5999, 4.0),
 (6134, 4.0),
 (6136, 4.0),
 (6755, 4.0),
 (9, 3.8333333333333335),
 (84, 3.6666666666666665),
 (6, 3.5),
 (35, 3.5),
 (55, 3.5),
 (133, 3.5),
 (176, 3.5),
 (217, 3.5),
 (254, 3.5),
 (337, 3.5),
 (431, 3.5),
 (474, 3.5),
 (957, 3.5),
 (1257, 3.5),
 (1290, 3.5

In [88]:
left_values_list1 = set([item[0] for item in enu_re])
left_values_list2 = set([item[0] for item in cos_re])


intersection = left_values_list1.intersection(left_values_list2)

In [90]:
for i in intersection:
    print(i)

277
5901
